In [5]:
# import spoof_detection
import numpy as np
import pandas as pd

import os
import io
import PIL
import dlib
import torch
import requests
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from scipy.misc import imresize, imread
detector = dlib.get_frontal_face_detector()


import os
import time
import torch

import os
import random
import numpy as np
from torch.utils.data import Dataset
from skimage.color import rgb2hsv, hsv2rgb
from scipy.misc import imresize
from torch.utils.data.sampler import WeightedRandomSampler
import torch.optim as optim
from sklearn.metrics import log_loss, roc_auc_score
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
# from .test_internal import heatmap, test_network
# from .utils.networks import *
# from .utils.datasets import Internal

In [3]:
model_list = ['test', 'test__', 'test_b256_256']

datadir = ''
savedir = 'results/savedir'
evaluate_from = False

test_name = 'test'

def load_checkpoint(test_name):
    if evaluate_from:
        print("Evaluating from model: ", evaluate_from)
        model_filename = evaluate_from
    else:
        model_dir = os.path.join(os.path.join(savedir, test_name), 'save_models')
        latest_filename = os.path.join(model_dir, 'latest.txt')
        if os.path.exists(latest_filename):
            with open(latest_filename, 'r') as fin:
                model_filename = fin.readlines()[0].strip()
        else:
            return None
    print("=> loading checkpoint '{}'".format(model_filename))
    state = torch.load(model_filename)
    print("=> loaded checkpoint '{}'".format(model_filename))
    return state

model = load_checkpoint(test_name)

if model is not None:
            args.start_epoch = checkpoint['epoch'] + 1
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])

In [6]:
SIZE = 128
PATCH_SIZE = 32
RESOLUTION = 32
# MODEL_LOCATION = os.path.join('utils', 'network_patchnormlr3')
MODEL_LOCATION = '/media/dataserver/workspace/blanca/MSDNet-GCN/results/savedir/test/save_models/model_best.pth.tar'

# load model and set to evaluation mode
model = torch.load(MODEL_LOCATION).cuda()
model.eval()
model.train(False)

AttributeError: Can't get attribute '_rebuild_tensor_v2' on <module 'torch._utils' from '/root/miniconda3/lib/python3.6/site-packages/torch/_utils.py'>

In [ ]:
def detect_face(img):
    ''' Detect faces in the image and return a crop of the largest face. '''
    # detect faces
    dets, conf, _ = detector.run(img)
    # filter detections with conf <= 0
    dets = [d for d, c in zip(dets,conf) if c > 0]
    # if at least one face was detected
    if len(dets) > 0:
        # calculate face sizes per detection
        size = [max(d.bottom(), 0) - max(d.top(), 0) + max(d.right(), 0) - max(d.left(), 0) for d in dets]
        # select detection corresponding with largest face
        det = dets[np.argmax(size)]
        # get face positions
        top, bot, left, right = max(det.top(), 0), max(det.bottom(), 0), max(det.left(), 0), max(det.right(), 0)
        # return cropped image
        return img[top:bot, left:right]
    return None

def create_heatmap(face_crop):
    ''' Select uniformly sampled patches from the face_crop and run them through the network. '''
    face_crop = face_crop.transpose(2, 0, 1)
    c, h, w = face_crop.shape
    # create a batch with uniformly sampled patches
    hr = float(h - PATCH_SIZE) / (RESOLUTION - 1)
    wr = float(w - PATCH_SIZE) / (RESOLUTION - 1)
    batch = np.zeros((RESOLUTION**2, 3, PATCH_SIZE, PATCH_SIZE))
    for y in range(RESOLUTION):
        for x in range(RESOLUTION):
            patch = face_crop[:, int(y * hr):int(y * hr + PATCH_SIZE), int(x * wr):int(x * wr + PATCH_SIZE)]
            patch = (patch - np.mean(patch)) / (np.std(patch) + 1e-8)
            batch[y * RESOLUTION + x, :, :, :] = patch
    # network makes a prediction per patch
    inputs = Variable(torch.from_numpy(batch).float(), volatile=True).cuda()
    outputs = F.softmax(model(inputs), dim=1).data.cpu().numpy()
    #score = np.mean(outputs, 0)[0]
    #outputs = model(inputs).data.cpu().numpy()
    #score = np.mean(outputs, 0)
    #score = np.exp(score)[0] / sum(np.exp(score)) 
    return score

def predict(image):
    ''' Turns input image into a spoof attempt prediction. '''
    # 1. download image
    #url = 'https://imago.onfido.com/api/{}/{}/download'.format('live_photos', str(photo_id))
    #im_response = requests.get(url)
    #try:
    #    image = imread(io.BytesIO(im_response.content))
    #except:
    #    return 'failed: reading image'
    # filter greyscale images
    if len(image.shape) is not 3 or image.shape[2] < 3:
        return 'failed: image shape'
    # remove possible alpha channel
    if image.shape[2] > 3: 
        image = image[:,:,:3]
    # get exit and rotate
    #img = PIL.Image.open(io.BytesIO(im_response.content))
    #if hasattr(img, '_getexif') and img._getexif() is not None and 274 in img._getexif():
    #    orientation = img._getexif()[274]
    #    if orientation in [3,4]:
    #        image = np.rot90(image, 2)
    #    if orientation in [5,6]:
    #        image = np.rot90(image, 3)
    #    if orientation in [7,8]:
    #        image = np.rot90(image, 1)
    # detect face in image
    face_crop = detect_face(image)
    if face_crop is None:
        return 'failed: face detection'
    # resize image
    face_crop = imresize(face_crop, (SIZE, SIZE))
    # spoof prediction
    heatmap = create_heatmap(face_crop)
    return heatmap
    # transform heatmap into a spoof prediction (1.0 is spoof attack)
    spoof_score = heatmap
    # also return the most likely class
    #most_likely_class = ['genuine sample', 'picture attack', 'screen attack', 'document attack'][np.argmax(np.mean(heatmap, axis=(0, 1)))]
    return spoof_score


In [8]:
# load unique warning and passed live_photo_ids
warning_ids = np.load('../warning/warning_ids.npy')
passed_ids = np.load('../passed/passed_ids.npy')
all_ids = list(set(np.concatenate((warning_ids, passed_ids))))
all_ids.sort()

FileNotFoundError: [Errno 2] No such file or directory: '../warning/warning_ids.npy'

In [5]:
print len(set(warning_ids)), len(set(passed_ids))
print len(set(warning_ids)) + len(set(passed_ids))
print len(set(np.concatenate((warning_ids, passed_ids))))

521 4479 830
5830
5829


In [7]:
# create table
table = pd.DataFrame(np.zeros((5000,4)), columns=['live_photo_id', 'manual_label', 'spoof_label', 'spoof_score'])
table['live_photo_id'] = all_ids
table['manual_label'] = table['manual_label'].apply(str)
table['spoof_label'] = table['spoof_label'].apply(str)
table['spoof_label'] = 'None'
table['jonne_label'] = ''
# loop through table and set manual_label to warning or passed
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    if photo_id in warning_ids: table.set_value(index, 'manual_label', 'warning')
    if photo_id in passed_ids: table.set_value(index, 'manual_label', 'passed')

NameError: name 'all_ids' is not defined

In [6]:
# loop through table and set spoof_detection to the predicted spoof score using spoof_detection
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    prediction = spoof_detection.predict(photo_id)
    
    if type(prediction) is str:
        table.set_value(index, 'spoof_label', prediction)
    else:
        table.set_value(index, 'spoof_score', prediction)
        if prediction < 0.925:
            table.set_value(index, 'spoof_label', 'spoof')
        else:
            table.set_value(index, 'spoof_label', 'passed')
        
    print (index)

NameError: name 'table' is not defined

In [6]:
#writer = pd.ExcelWriter('anti-spoof-patch-based.xlsx')
#table.to_excel(writer,'Sheet1')
#writer.save()

In [7]:
# get labeled spoofs
import os
folder = '../warning/sorted'
folders = ['gen', 'pop', 'pod', 'pos', 'etc', 'unknown']
labels = ['genuine selfie', 'picture attack', 'document attack', 'screen attack', 'camera defect', 'uncertain spoof']
def check_folder(live_id):
    for sub, label in zip(folders, labels):
        if os.path.exists(os.path.join(folder,sub,str(live_id)+'.png')):
            return label

In [8]:
# loop through table
for index, row in table.iterrows():
    photo_id = row['live_photo_id']
    label = check_folder(photo_id)
    table.set_value(index, 'jonne_label', label)
    

In [9]:
print table

      live_photo_id manual_label             spoof_label  spoof_score  \
0            186370       passed                   spoof     0.996201   
1            313856      warning                  passed     0.002434   
2            439505       passed                   spoof     0.998563   
3            549714      warning                  passed     0.062495   
4            558681      warning                  passed     0.002560   
5            578999      warning                  passed     0.000165   
6            606128      warning                  passed     0.011200   
7            606469      warning                  passed     0.000093   
8            606763      warning                   spoof     0.941019   
9            607571      warning                  passed     0.007935   
10           607616      warning                  passed     0.004856   
11           607829      warning                  passed     0.076798   
12           608191      warning                  p

In [10]:
writer = pd.ExcelWriter('anti-spoof-patch-results-patchnormlr3-averagesoftmax.xlsx')
table.to_excel(writer,'Sheet1')
writer.save()